<a href="https://colab.research.google.com/github/junsookim1/Everday_Practice/blob/master/Koyang_Bike_Data(%EC%9D%BC_%EB%8B%A8%EC%9C%84_%EA%B5%AC%EB%B6%84_%EC%9D%B4%EB%8F%99%EA%B1%B0%EB%A6%AC%2C%EC%8B%9C%EA%B0%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import missingno as msno
from haversine import haversine

In [ ]:
df=pd.read_csv('01.운영이력_ver1(한글컬럼, 이용거리).csv')

/home/jun/anaconda3/envs/jun/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142688 entries, 0 to 3142687
Data columns (total 14 columns):
 #   Column      Dtype  
---  ------      -----  
 0   대여 번호       int64  
 1   대여 시간       object 
 2   대여 스테이션 번호  int64  
 3   대여 거치대 번호   int64  
 4   반납 시간       object 
 5   반납 스테이션 번호  int64  
 6   반납 거치대 번호   int64  
 7   추정 이동 거리    float64
 8   회원 구분       object 
 9   회원 번호       int64  
 10  비회원 번호      float64
 11  자전거 번호      object 
 12  관제 반납       float64
 13  이용 시간(s)    float64
dtypes: float64(4), int64(6), object(4)
memory usage: 335.7+ MB


In [ ]:
df['대여 시간'] =  pd.to_datetime(df['대여 시간'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df['반납 시간'] =  pd.to_datetime(df['반납 시간'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142688 entries, 0 to 3142687
Data columns (total 14 columns):
 #   Column      Dtype         
---  ------      -----         
 0   대여 번호       int64         
 1   대여 시간       datetime64[ns]
 2   대여 스테이션 번호  int64         
 3   대여 거치대 번호   int64         
 4   반납 시간       datetime64[ns]
 5   반납 스테이션 번호  int64         
 6   반납 거치대 번호   int64         
 7   추정 이동 거리    float64       
 8   회원 구분       object        
 9   회원 번호       int64         
 10  비회원 번호      float64       
 11  자전거 번호      object        
 12  관제 반납       float64       
 13  이용 시간(s)    float64       
dtypes: datetime64[ns](2), float64(4), int64(6), object(2)
memory usage: 335.7+ MB


In [ ]:
df.set_index('대여 시간',inplace=True)

In [ ]:
df

,대여 번호,대여 스테이션 번호,대여 거치대 번호,반납 시간,반납 스테이션 번호,반납 거치대 번호,추정 이동 거리,회원 구분,회원 번호,비회원 번호,자전거 번호,관제 반납,이용 시간(s)
대여 시간,,,,,,,,,,,,,
2017-01-01 00:00:41,15945541,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,791.0
2017-01-01 00:01:03,15945542,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,2961.0
2017-01-01 00:01:50,15945543,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,3600.0
2017-01-01 00:02:09,15945544,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN,829.0
2017-01-01 00:03:32,15945545,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN,912.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:52:39,21718608,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN,377.0
2019-12-31 23:53:59,21718610,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN,240.0
2019-12-31 23:55:27,21718611,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN,657.0


In [ ]:
df.resample(rule='D')

In [ ]:
df_time_mean = df.resample(rule='D').mean()
df_time_mean = df_time_mean.loc[:,['추정 이동 거리','이용 시간(s)']]
df_time_mean

,추정 이동 거리,이용 시간(s)
대여 시간,,
2017-01-01,9533.028766,1673.962457
2017-01-02,10088.323926,1435.165138
2017-01-03,9267.804818,1466.927812
2017-01-04,9712.572571,1365.047398
2017-01-05,9465.182827,1325.515328
...,...,...
2019-12-27,2982.797459,1419.180889
2019-12-28,3227.851692,1613.333754
2019-12-29,3673.012311,1429.379338


In [ ]:
df_time_sum = df.resample(rule='D').sum()
df_time_sum = df_time_sum.loc[:,['추정 이동 거리','이용 시간(s)']]
df_time_sum

,추정 이동 거리,이용 시간(s)
대여 시간,,
2017-01-01,12593131.0,2452355.0
2017-01-02,15727697.0,2502928.0
2017-01-03,15004576.0,2621400.0
2017-01-04,16997002.0,2649557.0
2017-01-05,16204393.0,2507875.0
...,...,...
2019-12-27,4226624.0,2361517.0
2019-12-28,4483486.0,2557134.0
2019-12-29,3878701.0,1771001.0


In [ ]:
df_1 = pd.merge(left = df_time_mean, right = df_time_sum, left_index = True, right_index = True, how = "inner")
df_1.columns = ['추정 이동 거리(평균)','이용 시간(s)(평균)','추정 이동 거리(합)','이용 시간(s)(합)']
df_1

,추정 이동 거리(평균),이용 시간(s)(평균),추정 이동 거리(합),이용 시간(s)(합)
대여 시간,,,,
2017-01-01,9533.028766,1673.962457,12593131.0,2452355.0
2017-01-02,10088.323926,1435.165138,15727697.0,2502928.0
2017-01-03,9267.804818,1466.927812,15004576.0,2621400.0
2017-01-04,9712.572571,1365.047398,16997002.0,2649557.0
2017-01-05,9465.182827,1325.515328,16204393.0,2507875.0
...,...,...,...,...
2019-12-27,2982.797459,1419.180889,4226624.0,2361517.0
2019-12-28,3227.851692,1613.333754,4483486.0,2557134.0
2019-12-29,3673.012311,1429.379338,3878701.0,1771001.0


In [ ]:
df_1.to_csv('일별 이동거리_시간(평균,합).csv') # 저장
df_1 = pd.read_csv('일별 이동거리_시간(평균,합).csv')# 불러오기